In [150]:
import movingpandas as mpd
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pyproj import CRS
from keplergl import KeplerGl
from datetime import datetime, timedelta

import ipywidgets
ipywidgets.__version__

# 1. Create layer with geopandas

In [151]:
# Reading file
file = 'ais_202209.csv'
data = pd.read_csv(file, sep=';')

In [152]:
data['mmsi']=data['mmsi'].apply(str)

In [176]:
data['mmsi'] = 'mmsi'+data['mmsi']

In [153]:
lon_mean = data['lon'].mean()
lat_mean = data['lat'].mean()

In [177]:
# Create a geometry column as point geometry
data['geometry'] = [Point(long, lat) for long, lat in zip(data['lon'].to_list(), data['lat'].to_list())]

/Users/bonvie/opt/anaconda3/envs/movingpandas/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [178]:
# Create a Geodataframe. Be aware it is CRS 4326 WGS84
geodata = gpd.GeoDataFrame(data, crs = CRS.from_epsg('4326'))

In [179]:
# Create timestamp with Pandas datetime. Include None as timezone.
geodata['t'] = pd.to_datetime(geodata['date_time_utc']).dt.tz_localize(None)
geodata = geodata.set_index('t')
geodata.head()

,mmsi,imo_nr,length,date_time_utc,lon,lat,sog,cog,true_heading,nav_status,message_nr,geometry
t,,,,,,,,,,,,
2022-09-13 10:51:08,mmsi219164000,7926409,86,2022-09-13 10:51:08,8.78925,58.0993,8.2,284.1,284,0,1,POINT (8.78925 58.09930)
2022-09-13 10:51:19,mmsi219164000,7926409,86,2022-09-13 10:51:19,8.78850,58.0994,8.0,283.9,284,0,1,POINT (8.78850 58.09940)
2022-09-13 10:51:29,mmsi219164000,7926409,86,2022-09-13 10:51:29,8.78780,58.0995,8.1,285.9,283,0,1,POINT (8.78780 58.09950)
2022-09-13 10:51:39,mmsi219164000,7926409,86,2022-09-13 10:51:39,8.78712,58.0996,8.1,284.6,284,0,1,POINT (8.78712 58.09960)
2022-09-13 10:51:49,mmsi219164000,7926409,86,2022-09-13 10:51:49,8.78649,58.0997,8.1,283.6,285,0,1,POINT (8.78649 58.09970)


## 2. Stop detection with Movingpandas

In [180]:
# Create a Trajectory Collection with Movingpandas
traj_collection = mpd.TrajectoryCollection(geodata, 'mmsi')

In [181]:
# Define parameters in Hours and Search radio in meters
Hours = 0
SearchRadio = 100

# Stop detection
stops = mpd.TrajectoryStopDetector(traj_collection).get_stop_segments(min_duration=timedelta(hours=Hours), max_diameter=SearchRadio)

/Users/bonvie/opt/anaconda3/envs/movingpandas/lib/python3.10/site-packages/movingpandas/geometry_utils.py:150: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  if len(geom) == 2:
/Users/bonvie/opt/anaconda3/envs/movingpandas/lib/python3.10/site-packages/movingpandas/geometry_utils.py:151: ShapelyDeprecationWarning: __getitem__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  return _measure_distance(geom[0], geom[1], spherical)


In [182]:
# Create a new Geodataframe and define geometry column
stops_start = gpd.GeoDataFrame(columns = ['geometry'])
stops_start = stops_start.set_geometry('geometry')

# Add the ID of each stop track and define it as index
stops_start['stop_id'] = [track.id for track in stops.trajectories]
stops_start = stops_start.set_index('stop_id')

# Iteration over the Stop Trajectories
for stoptrack in stops.trajectories:
    
    # add stop duration in hours
    stops_start.at[stoptrack.id, 'duration_h'] = stoptrack.get_duration().total_seconds()/3600
    
    # add length
    stops_start.at[stoptrack.id, 'length_m'] = stoptrack.get_length()
    
    # add vessel name
    stops_start.at[stoptrack.id, 'vessel'] = stoptrack.id.split('_')[0]
    
    # add datetime
    stops_start.at[stoptrack.id, 'datetime'] = stoptrack.id.split('_')[1]
    
    # geometry with start point
    stops_start.at[stoptrack.id, 'geometry'] = stoptrack.get_start_location()

stops_start.head()

,geometry,duration_h,length_m,vessel,datetime
stop_id,,,,,
mmsi219164000_2022-09-13 10:51:08,POINT (8.78925 58.09930),0.005833,88.362093,mmsi219164000,2022-09-13 10:51:08
mmsi219164000_2022-09-13 10:51:49,POINT (8.78649 58.09970),0.005278,84.962889,mmsi219164000,2022-09-13 10:51:49
mmsi219164000_2022-09-13 10:52:29,POINT (8.78368 58.10010),0.005278,81.934898,mmsi219164000,2022-09-13 10:52:29
mmsi219164000_2022-09-13 10:53:08,POINT (8.78107 58.10060),0.005833,87.609693,mmsi219164000,2022-09-13 10:53:08
mmsi219164000_2022-09-13 10:53:48,POINT (8.77823 58.10090),0.005556,95.049264,mmsi219164000,2022-09-13 10:53:48


In [146]:
stops.trajectories

[Trajectory 219164000_2022-09-13 10:51:08 (2022-09-13 10:51:08 to 2022-09-13 10:51:29) | Size: 3 | Length: 88.4m
 Bounds: (8.7878, 58.0993, 8.78925, 58.0995)
 LINESTRING (8.78925 58.0993, 8.7885 58.0994, 8.7878 58.0995),
 Trajectory 219164000_2022-09-13 10:51:49 (2022-09-13 10:51:49 to 2022-09-13 10:52:08) | Size: 3 | Length: 85.0m
 Bounds: (8.7851, 58.0997, 8.78649, 58.0999)
 LINESTRING (8.78649 58.0997, 8.78573 58.0998, 8.7851 58.0999),
 Trajectory 219164000_2022-09-13 10:52:29 (2022-09-13 10:52:29 to 2022-09-13 10:52:48) | Size: 3 | Length: 81.9m
 Bounds: (8.78242, 58.1001, 8.78368, 58.1004)
 LINESTRING (8.78368 58.1001, 8.78302 58.1003, 8.78242 58.1004),
 Trajectory 219164000_2022-09-13 10:53:08 (2022-09-13 10:53:08 to 2022-09-13 10:53:29) | Size: 3 | Length: 87.6m
 Bounds: (8.77961, 58.1006, 8.78107, 58.1007)
 LINESTRING (8.78107 58.1006, 8.78039 58.1007, 8.77961 58.1007),
 Trajectory 219164000_2022-09-13 10:53:48 (2022-09-13 10:53:48 to 2022-09-13 10:54:08) | Size: 3 | Length: 95

In [183]:
# Reset indexes
stops_start = stops_start.reset_index(drop=True)
stops_start.head()

,geometry,duration_h,length_m,vessel,datetime
0,POINT (8.78925 58.09930),0.005833,88.362093,mmsi219164000,2022-09-13 10:51:08
1,POINT (8.78649 58.09970),0.005278,84.962889,mmsi219164000,2022-09-13 10:51:49
2,POINT (8.78368 58.10010),0.005278,81.934898,mmsi219164000,2022-09-13 10:52:29
3,POINT (8.78107 58.10060),0.005833,87.609693,mmsi219164000,2022-09-13 10:53:08
4,POINT (8.77823 58.10090),0.005556,95.049264,mmsi219164000,2022-09-13 10:53:48


In [163]:
stops_start.dtypes

geometry      geometry
duration_h     float64
length_m       float64
vessel          object
datetime        object
dtype: object

In [184]:
# Reset indexes
geodata= geodata.reset_index(drop=True)
geodata.head()

,mmsi,imo_nr,length,date_time_utc,lon,lat,sog,cog,true_heading,nav_status,message_nr,geometry
0,mmsi219164000,7926409,86,2022-09-13 10:51:08,8.78925,58.0993,8.2,284.1,284,0,1,POINT (8.78925 58.09930)
1,mmsi219164000,7926409,86,2022-09-13 10:51:19,8.78850,58.0994,8.0,283.9,284,0,1,POINT (8.78850 58.09940)
2,mmsi219164000,7926409,86,2022-09-13 10:51:29,8.78780,58.0995,8.1,285.9,283,0,1,POINT (8.78780 58.09950)
3,mmsi219164000,7926409,86,2022-09-13 10:51:39,8.78712,58.0996,8.1,284.6,284,0,1,POINT (8.78712 58.09960)
4,mmsi219164000,7926409,86,2022-09-13 10:51:49,8.78649,58.0997,8.1,283.6,285,0,1,POINT (8.78649 58.09970)


In [185]:
geodata.dtypes

mmsi               object
imo_nr              int64
length              int64
date_time_utc      object
lon               float64
lat               float64
sog               float64
cog               float64
true_heading        int64
nav_status          int64
message_nr          int64
geometry         geometry
dtype: object

## 3. Point map visualisation with KeplerGI

In [186]:
# Create KeplerGI instance
m = KeplerGl(height=600)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [187]:
m.add_data(stops_start, 'stops')

In [188]:
m.add_data(geodata, 'trajectories')

In [189]:
geodata.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 149325 entries, 0 to 149324
Data columns (total 12 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   mmsi           149325 non-null  object 
 1   imo_nr         149325 non-null  int64  
 2   length         149325 non-null  int64  
 3   date_time_utc  149325 non-null  object 
 4   lon            149325 non-null  float64
 5   lat            149325 non-null  float64
 6   sog            149325 non-null  float64
 7   cog            149325 non-null  float64
 8   true_heading   149325 non-null  int64  
 9   nav_status     149325 non-null  int64  
 10  message_nr     149325 non-null  int64  
 11  geometry       149325 non-null  object 
dtypes: float64(4), int64(5), object(3)
memory usage: 13.7+ MB


In [190]:
config={
  "version": "v1",
  "config": {
    "visState": {
      "filters": [
        {
          "dataId": [
            "trajectories"
          ],
          "id": "ci8tl8aip",
          "name": [
            "date_time_utc"
          ],
          "type": "timeRange",
          "value": [
            1376524868000,
            1376747899999.9995
          ],
          "enlarged": True,
          "plotType": "histogram",
          "animationWindow": "free",
          "yAxis": None
        }
      ],
      "layers": [
        {
          "id": "w0gvb3l",
          "type": "geojson",
          "config": {
            "dataId": "trajectories",
            "label": "trajectories",
            "color": [
              136,
              87,
              44
            ],
            "columns": {
              "geojson": "geometry"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.4,
              "strokeOpacity": 0.8,
              "thickness": 0.5,
              "strokeColor": None,
              "colorRange": {
                "name": "ColorBrewer Dark2-3",
                "type": "qualitative",
                "category": "ColorBrewer",
                "colors": [
                  "#1b9e77",
                  "#d95f02",
                  "#7570b3"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 8,
              "sizeRange": [
                0,
                10
              ],
              "radiusRange": [
                0,
                50
              ],
              "heightRange": [
                0,
                500
              ],
              "elevationScale": 5,
              "stroked": False,
              "filled": True,
              "enable3d": False,
              "wireframe": False
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "mmsi",
              "type": "string"
            },
            "colorScale": "ordinal",
            "sizeField": None,
            "sizeScale": "linear",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "heightField": None,
            "heightScale": "linear",
            "radiusField": None,
            "radiusScale": "linear"
          }
        },
        {
          "id": "nsjcvzk",
          "type": "geojson",
          "config": {
            "dataId": "stops",
            "label": "stops",
            "color": [
              18,
              147,
              154
            ],
            "columns": {
              "geojson": "geometry"
            },
            "isVisible": True,
            "visConfig": {
              "opacity": 0.3,
              "strokeOpacity": 0.8,
              "thickness": 0.1,
              "strokeColor": [
                38,
                26,
                16
              ],
              "colorRange": {
                "name": "ColorBrewer Dark2-3",
                "type": "qualitative",
                "category": "ColorBrewer",
                "colors": [
                  "#1b9e77",
                  "#d95f02",
                  "#7570b3"
                ]
              },
              "strokeColorRange": {
                "name": "Global Warming",
                "type": "sequential",
                "category": "Uber",
                "colors": [
                  "#5A1846",
                  "#900C3F",
                  "#C70039",
                  "#E3611C",
                  "#F1920E",
                  "#FFC300"
                ]
              },
              "radius": 10,
              "sizeRange": [
                0,
                10
              ],
              "radiusRange": [
                0,
                50
              ],
              "heightRange": [
                0,
                500
              ],
              "elevationScale": 5,
              "stroked": False,
              "filled": True,
              "enable3d": False,
              "wireframe": False
            },
            "hidden": False,
            "textLabel": [
              {
                "field": None,
                "color": [
                  255,
                  255,
                  255
                ],
                "size": 18,
                "offset": [
                  0,
                  0
                ],
                "anchor": "start",
                "alignment": "center"
              }
            ]
          },
          "visualChannels": {
            "colorField": {
              "name": "bird",
              "type": "string"
            },
            "colorScale": "ordinal",
            "sizeField": None,
            "sizeScale": "linear",
            "strokeColorField": None,
            "strokeColorScale": "quantile",
            "heightField": None,
            "heightScale": "linear",
            "radiusField": {
              "name": "duration_h",
              "type": "real"
            },
            "radiusScale": "sqrt"
          }
        }
      ],
      "interactionConfig": {
        "tooltip": {
          "fieldsToShow": {
            "stops": [
              {
                "name": "duration_h",
                "format": None
              },
              {
                "name": "length_m",
                "format": None
              },
              {
                "name": "vessel",
                "format": None
              },
              {
                "name": "datetime",
                "format": None
              }
            ],
            "trajectories": [
              {
                "name": "imo_nr",
                "format": None
              },
              {
                "name": "length",
                "format": None
              },
              {
                "name": "date_time_utc",
                "format": None
              },
              {
                "name": "sog",
                "format": None
              },
              {
                "name": "cog",
                "format": None
              },
              {
                "name": "true_heading",
                "format": None
              },
              {
                "name": "nav_status",
                "format": None
              },    
              {
                "name": "message_nr",
                "format": None
              }             
            ]
          },
          "compareMode": False,
          "compareType": "absolute",
          "enabled": True
        },
        "brush": {
          "size": 0.5,
          "enabled": False
        },
        "geocoder": {
          "enabled": False
        },
        "coordinate": {
          "enabled": False
        }
      },
      "layerBlending": "normal",
      "splitMaps": [],
      "animationConfig": {
        "currentTime": None,
        "speed": 1
      }
    },
    "mapState": {
      "bearing": 0,
      "dragRotate": False,
      "latitude": lat_mean,
      "longitude": lon_mean,
      "pitch": 0,
      "zoom": 3.249643007871771,
      "isSplit": False
    },
    "mapStyle": {
      "styleType": "dark",
      "topLayerGroups": {},
      "visibleLayerGroups": {
        "label": True,
        "road": True,
        "border": False,
        "building": True,
        "water": True,
        "land": True,
        "3d building": False
      },
      "threeDBuildingColor": [
        9.665468314072013,
        17.18305478057247,
        31.1442867897876
      ],
      "mapStyles": {}
    }
  }
}

In [191]:
m.save_to_html(file_name='index_norge.html', config=config)

Map saved to index_norge.html!


In [33]:
lon_mean

7.080952030202578